<a href="https://colab.research.google.com/github/LFBBMRML/Generative_Adversarial_Network/blob/main/original_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
tf.__version__
%load_ext tensorboard
from tensorflow.keras.utils import plot_model
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.models import load_model
from google.colab import files
import matplotlib.pyplot as plt
import datetime
import time as zeit
from datetime import time

from keras import layers
%matplotlib inline

In [ ]:
def define_discriminator(in_shape=(32, 32, 3)):
  D = Sequential()

  D.add(Conv2D(64, (3, 3), padding = 'same', input_shape=in_shape))
  D.add(LeakyReLU(alpha=0.2))

  D.add(Conv2D(128, (3, 3), strides=(2,2), padding='same'))
  D.add(LeakyReLU(alpha=0.2))

  D.add(Conv2D(128, (3, 3), strides=(2,2), padding = 'same'))
  D.add(LeakyReLU(alpha=0.2))

  D.add(Conv2D(256, (3, 3), strides=(2,2), padding = 'same'))
  D.add(LeakyReLU(alpha=0.2))

  D.add(Flatten())
  D.add(Dropout(0.4))
  D.add(Dense(1, activation = 'sigmoid'))

  opt = optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
  D.compile(loss='binary_crossentropy', optimizer=opt, metrics = ['accuracy'])

  return D

In [ ]:
def define_generator(latent_dim):
  G = Sequential()

  G.add(Dense(256 * 4 * 4, input_dim=latent_dim))
  G.add(LeakyReLU(alpha=0.2))
  G.add(layers.Reshape((4, 4, 256)))

  G.add(Conv2DTranspose(128, (4, 4), strides=(2,2), padding='same'))
  G.add(LeakyReLU(alpha=0.2))

  G.add(Conv2DTranspose(128, (4, 4), strides=(2,2), padding='same'))
  G.add(LeakyReLU(alpha=0.2))

  G.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
  G.add(LeakyReLU(alpha=0.2))

  G.add(Conv2D(3, (3 ,3), activation='tanh', padding='same'))
  
  return G

In [ ]:
def define_gan(g_model, d_model):
  d_model.trainable = False

  model = Sequential()
  model.add(g_model)
  model.add(d_model)

  opt = optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss = 'binary_crossentropy', optimizer=opt)
  return model

In [ ]:
def load_real_samples():
  (train, _), (_,_) = tf.keras.datasets.cifar10.load_data()
  images = train.astype('float32')
  images = (images - 127.5) / 127.5
  return images

In [ ]:
def generate_real_samples(dataset, n_samples):
  ix = randint(0, dataset.shape[0], n_samples)
  X = dataset[ix]
  y = ones((n_samples, 1))

  return X, y

In [ ]:
def generate_latent_points(latent_dim, n_samples):
  x_input = randn(latent_dim * n_samples)
  x_input = x_input.reshape(n_samples, latent_dim)
  return x_input

In [ ]:
def generate_fake_samples(g_model, latent_dim, n_samples):
  x_input = generate_latent_points(latent_dim, n_samples)
  X = g_model.predict(x_input)
  y = zeros((n_samples, 1))
  return X, y

In [ ]:
def save_plot(examples, epoch, n=6):
  examples = (examples + 1) / 2.0

  for i in range(n*n):
    plt.subplot(n, n, i+1)
    plt.axis('off')
    plt.imshow(examples[i])
  filename_plot = 'generated_plot_e%03d.png' % (epoch+1)
  plt.savefig(filename_plot)
  #files.download(filename_plot)
  plt.close()

In [ ]:
def summarize_performance(epoch, gan_model, g_model, d_model, dataset, latent_dim, n_samples=150):
  X_real, y_real = generate_real_samples(dataset, n_samples)
  _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)

  X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
  _, acc_fake = d_model.evaluate(X_fake, y_fake, verbose=0)

  print('>Accuracy real: %.0f%%, fake:%.0f%%' % (acc_real*100, acc_fake*100))

  save_plot(X_fake, epoch)  
  filename = 'Modell_Daten_%03d.h5' % (epoch+1)
  g_model.save(filename)

In [ ]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=64):
  bat_per_epo = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch/2)
  ges_time = []
  

  for i in range(n_epochs):
    start = zeit.time()
    for j in range(bat_per_epo):
      X_real, y_real = generate_real_samples(dataset, half_batch)
      d_loss1, _ = d_model.train_on_batch(X_real, y_real)

      X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)

      X_gan = generate_latent_points(latent_dim, n_batch)
      y_gan = ones((n_batch, 1))
      g_loss = gan_model.train_on_batch(X_gan, y_gan)

      #print('>%d, %d/%d, d1=%.3f, d2=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
      #plot_model(d_model, to_file='original_Diskriminator.png', show_shapes=True, show_layer_names=True, rankdir='TB')
      #plot_model(g_model, to_file='original_Generator.png', show_shapes=True, show_layer_names=True, rankdir='TB')
      #plot_model(gan_model, to_file='original_GAN_Modell.png', show_shapes=True, show_layer_names=True, rankdir='TB')
    
    ges_time.append(zeit.time()-start)
    minuten = int((sum(ges_time))/60)
    sec = sum(ges_time)%60

    if (i+1) % 1 == 0:
      summarize_performance(i, gan_model, g_model, d_model, dataset, latent_dim)
    print("Die gesamte Dauer beträgt: " + str(minuten) + " Minuten und " + str(sec) + " Sekunde, für " + str(i+1) + " Epochen ")


In [ ]:
latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()

In [ ]:
train(g_model, d_model, gan_model, dataset, latent_dim)